In [2]:
import pandas as pd
import numpy as np
from src.utils import load_json
from collections import defaultdict

In [3]:
sp_ops_raw = load_json("../data/sp_ops/known_rxns_swissprot_enzymes_240310.json")
imt_ops = pd.read_csv("../data/sp_ops/JN3604IMT_rules.tsv", sep='\t')
min_ops = pd.read_csv("../data/sp_ops/minimal1224_all_uniprot.tsv", sep='\t')

In [4]:
id2seq = defaultdict(set)
for _, elt in sp_ops_raw.items():
    if elt['imt_rules'] and elt['enzymes']:
        for e in elt['enzymes']:
                id2seq[e[0]].add(e[1])

# Make sure only one seq per id
for k,v in id2seq.items():
     if len(v) > 1:
          print(k,v)

id2rule = defaultdict(set)
for _, elt in sp_ops_raw.items():
    if elt['imt_rules'] and elt['enzymes']:
        for e in elt['enzymes']:
            for r in elt['imt_rules']:
                 id2rule[e[0]].add(r)

print(set(id2rule.keys()) ^ set(id2seq.keys())) # Make sure these are the same

data = {'Entry':[], 'Label':[], 'Sequence':[]}
for id in id2seq.keys():
     data['Entry'].append(id)
     data['Label'].append(";".join(id2rule[id]))
     data['Sequence'].append(list(id2seq[id])[0])

# Save to sp_ops subdir

df = pd.DataFrame(data=data)
df.to_csv("../data/sp_ops/sp_ops.csv", index=False, sep='\t')
df.head()

set()


,Entry,Label,Sequence
0,A1Z1Q3,rule0310_1;rule0006_177;rule0007_187;rule0007_...,MYPSNKKKKVWREEKERLLKMTLEERRKEYLRDYIPLNSILSWKEE...
1,A0A009IHW8,rule0310_1;rule0311_1,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
2,A0A024B7W1,rule0010_65;rule0121_1;rule0011_51;rule0120_1,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...
3,A0A059TC02,rule0142_1;rule0143_2,MRSVSGQVVCVTGAGGFIASWLVKILLEKGYTVRGTVRNPDDPKNG...
4,A0A061AE05,rule0015_34;rule0014_29,MLTPRDENNEGDAMPMLKKPRYSSLSGQSTNITYQEHTISREERAA...


In [5]:
from src.utils import construct_sparse_adj_mat

y, idx_sample, idx_feature = construct_sparse_adj_mat("sp_ops")

Constructing sp_ops sparse adjacency matrix


In [6]:
y.shape

(3660, 2271)